In [12]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from crewai_tools import BaseTool, tool

# Load environment variables
load_dotenv()
# Get the API key from the environment
api_key = os.getenv("OPENAI_API_KEY")


llm = ChatOpenAI(
    model="gpt-4o",  
    api_key=api_key  
)

# Define the agent with prompt engineering capabilities
prompt_engineer = Agent(
    role='Prompt Engineer',
    goal='Enhance and optimize prompts iteratively to improve LLM performance.',
    backstory=(
        "You are a seasoned Prompt Engineer with deep expertise in natural language processing. "
        "Your primary skill is in fine-tuning prompts to maximize clarity, relevance, and effectiveness. "
        "You have a systematic approach to testing and refining prompts to achieve the best outcomes."
    ),
    memory=True,
    verbose=True,
    llm=llm
)


In [13]:
# Define tools
@tool("General Prompt")
def general_prompt(idea: str) -> str:
    """Generate a general prompt based on the given idea."""
    return f"Please provide information about {idea}."

@tool("No-Shot Prompt")
def no_shot_prompt(idea: str) -> str:
    """Generate a no-shot prompt based on the given idea."""
    return f"Without any examples, please explain or answer: {idea}"

@tool("Few-Shot Prompt")
def few_shot_prompt(idea: str) -> str:
    """Generate a few-shot prompt based on the given idea."""
    examples = [
        "Example 1: [Input] -> [Output]",
        "Example 2: [Input] -> [Output]",
        "Example 3: [Input] -> [Output]"
    ]
    return f"Given the following examples:\n{'\n'.join(examples)}\n\nNow, please address: {idea}"

@tool("Include-Exclude Prompt")
def include_exclude_prompt(idea: str) -> str:
    """Generate an include-exclude prompt based on the given idea."""
    return f"""
    Regarding {idea}:
    Include:
    - Relevant point 1
    - Relevant point 2
    Exclude:
    - Irrelevant point 1
    - Irrelevant point 2
    Please provide your response based on these guidelines.
    """

@tool("Chain-of-Thought Prompt")
def chain_of_thought_prompt(idea: str) -> str:
    """Generate a chain-of-thought prompt based on the given idea."""
    return f"""
    Let's approach {idea} step by step:
    1. First, consider...
    2. Then, analyze...
    3. Next, evaluate...
    4. Finally, conclude...
    Please provide your reasoning for each step.
    """

@tool("CoT Reflection Prompt")
def cot_reflection_prompt(idea: str) -> str:
    """Generate a chain-of-thought prompt with reflection based on the given idea."""
    return f"""
    Let's approach {idea} using a reflective chain of thought:
    1. Initial thoughts: ...
    2. Deeper analysis: ...
    3. Potential issues or contradictions: ...
    4. Revised perspective: ...
    5. Final conclusion: ...
    At each step, reflect on your reasoning and consider alternative viewpoints.
    """

In [14]:
# Define tasks
analyze_prompt_task = Task(
    description=(
        "Analyze the given prompt and choose the most suitable prompt technique for the task. "
        "Consider aspects such as clarity, relevance, and potential ambiguities. "
        "Provide a detailed explanation for each identified issue, referencing best practices in prompt engineering. "
        "Recommend the most appropriate prompt technique (e.g., Chain-of-Thought, Include-Exclude) and justify your choice.\n\n"
        "Prompt: {prompt}\n\n"
        "Your analysis should be thorough and actionable, offering specific suggestions for enhancing the prompt and selecting the best technique."
    ),
    expected_output='A comprehensive analysis of the prompt with detailed explanations, actionable suggestions for improvement, and a recommendation for the most suitable prompt technique.',
    agent=prompt_engineer
)

generate_prompt_task = Task(
    description=(
        "Generate a detailed and comprehensive prompt based on the analysis done, ensuring clarity, relevance, and adherence to the guidelines provided."
    ),
    expected_output='prompt that follows the decision made for which prompt technique to use',
    agent=prompt_engineer,
    tools=[general_prompt , no_shot_prompt , few_shot_prompt , include_exclude_prompt , chain_of_thought_prompt , cot_reflection_prompt]
)



In [15]:
crew = Crew(
    agents=[prompt_engineer],
    tasks=[analyze_prompt_task, generate_prompt_task],
    verbose=True
)

2024-09-22 15:09:09,740 - 8267676352 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [16]:
prompt = 'What is the capital of the moon?'

crew.kickoff({'prompt': prompt})

 [2024-09-22 15:09:36][DEBUG]: == Working Agent: Prompt Engineer
 [2024-09-22 15:09:36][INFO]: == Starting Task: Analyze the given prompt and choose the most suitable prompt technique for the task. Consider aspects such as clarity, relevance, and potential ambiguities. Provide a detailed explanation for each identified issue, referencing best practices in prompt engineering. Recommend the most appropriate prompt technique (e.g., Chain-of-Thought, Include-Exclude) and justify your choice.

Prompt: What is the capital of the moon?

Your analysis should be thorough and actionable, offering specific suggestions for enhancing the prompt and selecting the best technique.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.

Current Task: Analyze the given prompt and choose the most suitable prompt technique for the task. Consider aspects such as clarity, relevance, and potential ambiguities. Provide a deta

CrewOutput(raw='"Can you list some notable landmarks or features on the moon? Please exclude any references to political entities or capitals, as the moon does not have these.\n\nInclude:\n- Relevant point 1\n- Relevant point 2\n\nExclude:\n- Irrelevant point 1\n- Irrelevant point 2\n\nPlease provide your response based on these guidelines."', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Analyze the given prompt and choose the most suitable prompt technique for the task. Consider aspects such as clarity, relevance, and potential ambiguities. Provide a detailed explanation for each identified issue, referencing best practices in prompt engineering. Recommend the most appropriate prompt technique (e.g., Chain-of-Thought, Include-Exclude) and justify your choice.\n\nPrompt: What is the capital of the moon?\n\nYour analysis should be thorough and actionable, offering specific suggestions for enhancing the prompt and selecting the best technique.', name=None, expecte